# Local Invariant Features for Object Detection
## Computer Vision and Image Processing - Lab Session 6
### Prof: Luigi Di Stefano, luigi.distefano@unibo.it
### Tutor: Pierluigi Zama Ramirez, pierluigi.zama@unibo.it